# Make Synthetic Genome
Author : Mathieu Giguere \
Date : 14/08/2024 \
Brief : Strings together the genes of interest of some species into a singular synthetic genome. \
Dependencies :
- "gene-species.csv" a csv file with 2 columns, gene-ids and species.
- Packages: re, pandas. 
- Reference genome Fasta files
- corresponding GFF files 

## Pseudocode
1. Read genome files (fasta) -> big string.
2. Read gffs files -> then use regex to extract the coordinates of the genes of interest.
3. For each genome, for each gene of interest, extract its dna sequence in the appropriate genome file.
4. Append each string into 1 genome with spacers.
5. Write synthetic genome file in fasta format.

## Import packages, set constants

In [1]:
import re
import pandas as pd

In [2]:
# IMPORTANT: make sure these list have the species in the same order.  

species_genome_file_list = ["fasta_files/unzipped/Aspergillus_fumigatus.ASM265v1.dna.toplevel.fa",
                           "fasta_files/unzipped/C_albicans_SC5314_version_A22-s07-m01-r195_chromosomes.fasta",
                           "fasta_files/unzipped/C_auris_B8441_version_s01-m03-r08_chromosomes.fasta",
                           "fasta_files/unzipped/C_parapsilosis_CDC317_version_s01-m06-r03_chromosomes.fasta",
                           "fasta_files/unzipped/Candida_tropicalis.GCA000006335v3.dna.toplevel.fa",
                           "fasta_files/unzipped/Cryptococcus_neoformans.ASM9104v1.dna.toplevel.fa",
                           "fasta_files/unzipped/C_glabrata_CBS138_version_s05-m03-r06_chromosomes.fasta",
                           "fasta_files/unzipped/Pichia_kudriavzevii_gca_000764455.ASM76445v1.dna.toplevel.fa"]

species_list = ["Aspergillus fumigatus", "Candida albicans", "Candida auris", "Candida parapsilosis", "Candida tropicalis", "Cryptococcus neoformans", "Nasakeomyces glabrata", "Pichia kudriavzevii"]

gff_files_list = ["gff_files/Aspergillus_fumigatus.ASM265v1.59.gff3",
                 "gff_files/C_albicans_SC5314_version_A22-s07-m01-r195.gff",
                 "gff_files/C_auris_B8441_version_s01-m03-r08.gff",
                 "gff_files/C_parapsilosis_CDC317_version_s01-m06-r03.gff",
                 "gff_files/Candida_tropicalis.GCA000006335v3.59.gff3",
                 "gff_files/Cryptococcus_neoformans.ASM9104v1.59.gff3",
                "gff_files/C_glabrata_CBS138_version_s05-m03-r06.gff",
                 "gff_files/Pichia_kudriavzevii_gca_000764455.ASM76445v1.59.gff3"]

In [3]:
# Need to seperate gff files that come from different databases
# because they have different formatting, therefore we need to 
# implement different ways of parsing them.

gff_candid = ["gff_files/C_albicans_SC5314_version_A22-s07-m01-r195.gff",
              "gff_files/C_auris_B8441_version_s01-m03-r08.gff",
              "gff_files/C_glabrata_CBS138_version_s05-m03-r06.gff",
              "gff_files/C_parapsilosis_CDC317_version_s01-m06-r03.gff"]

gff_ensembl = ["gff_files/Aspergillus_fumigatus.ASM265v1.59.gff3",
               "gff_files/Candida_tropicalis.GCA000006335v3.59.gff3",
               "gff_files/Cryptococcus_neoformans.ASM9104v1.59.gff3",
               "gff_files/Pichia_kudriavzevii_gca_000764455.ASM76445v1.59.gff3"]

## Make dataframe that links the species to their genes of interest

In [4]:
df = pd.read_csv("gene-species.csv")

df = df.groupby('Species')['Gene ID'].apply(list).reset_index()

df.iloc[1]['Gene ID']

['C5_00660C_A',
 'C1_04770C_A',
 'C1_02420C_A',
 'CR_00850C_A',
 'C5_03390C_A',
 'C6_00620W_A',
 'C3_05920W_A',
 'C3_05220W_A',
 'C5_01840C_A',
 'C1_08460C_A',
 'C3_04890W_A',
 'C3_07860C_A',
 'C6_03170C_A',
 'C1_08590C_A',
 'C3_02220W_A',
 'C1_00800C_A',
 'C1_03780C_A',
 'C3_06850W_A',
 'CR_08780W_A',
 'CR_08800W_A',
 'C1_00710C_A']

## Get gff landmarks
It helps us find the right position of a gene in the reference genome.

In [5]:
list1 = []

for s, gff_file in enumerate(gff_files_list):
    
    with open(gff_file) as gff:
        gff_content = gff.read()

    landmark_dict = {}
    landmark = 0
    coord_list = []
    genome_file = species_genome_file_list[s]
    print(genome_file)
    
    if gff_file in gff_ensembl: # A. fumigatus, C. tropicalis, C. neoformans, P. kuridavzevii

        with open(genome_file) as gen:
            gen_content = gen.read()
        
        list_ids = []
        ids_tofind = re.findall(">.*? ", gen_content)
        
        for identificator_index in range(len(ids_tofind)):
            iden = re.sub(">", "", ids_tofind[identificator_index])
            iden = re.sub(" ", "", iden)
            list_ids.append(iden)
        
        for index, identificator in enumerate(list_ids):
            # look at gff files to understand the regex operations.
            landma_line = re.findall(f"##sequence-region   {identificator}.*\\n", gff_content)[0]
            coord = re.sub(".* ", "", landma_line)
            
            landmark += int(coord)
            coord_list.append(landmark)
            
            if index != 0:
                landmark_dict[identificator] = coord_list[index-1]
            else:
                landmark_dict[identificator] = 0

        list1.append(landmark_dict)
        
        
    else: # C. albicans, C. auris, N. glabrata, C. parapsilosis
        gff_landmarks = re.findall(r"^.*CGD\tchromosome.*|^.*CGD\tcontig.*", gff_content, flags=re.MULTILINE)
        
        for i, ld in enumerate(gff_landmarks):
            # look at gff files to understand the regex operations.
            index = re.findall(r"(?:^|(?:[.!?]\s))(\w+)", ld)[0]
            coord = re.findall("\t[0-9]{4}.+?\t", ld)[0]
            coord = re.sub("[^0-9]", "", coord)

            landmark += int(coord)
            coord_list.append(landmark)
            
            if i != 0:
                landmark_dict[index] = coord_list[i-1]
            else:
                landmark_dict[index] = 0
        
        list1.append(landmark_dict)


df["Landmarks"] = list1 

df

fasta_files/unzipped/Aspergillus_fumigatus.ASM265v1.dna.toplevel.fa
fasta_files/unzipped/C_albicans_SC5314_version_A22-s07-m01-r195_chromosomes.fasta
fasta_files/unzipped/C_auris_B8441_version_s01-m03-r08_chromosomes.fasta
fasta_files/unzipped/C_parapsilosis_CDC317_version_s01-m06-r03_chromosomes.fasta
fasta_files/unzipped/Candida_tropicalis.GCA000006335v3.dna.toplevel.fa
fasta_files/unzipped/Cryptococcus_neoformans.ASM9104v1.dna.toplevel.fa
fasta_files/unzipped/C_glabrata_CBS138_version_s05-m03-r06_chromosomes.fasta
fasta_files/unzipped/Pichia_kudriavzevii_gca_000764455.ASM76445v1.dna.toplevel.fa


,Species,Gene ID,Landmarks
0,Aspergillus fumigatus,"[AFUA_4G06890, AFUA_6G05140, AFUA_2G00320, AFU...","{'1': 0, '2': 4918979, '3': 9763451, '4': 1384..."
1,Candida albicans,"[C5_00660C_A, C1_04770C_A, C1_02420C_A, CR_008...","{'Ca22chr1A_C_albicans_SC5314': 0, 'Ca22chr1B_..."
2,Candida auris,"[B9J08_001448, B9J08_003737, B9J08_000964, B9J...","{'PEKT02000001_C_auris_B8441': 0, 'PEKT0200000..."
3,Candida parapsilosis,"[CPAR2_303740, CPAR2_105550, CPAR2_106400, CPA...","{'Contig005504_C_parapsilosis_CDC317': 0, 'Con..."
4,Candida tropicalis,"[CTRG_05283, CTRG_04480, CTRG_04661, CTRG_0268...","{'GG692418': 0, 'GG692395': 50304, 'GG692396':..."
5,Cryptococcus neoformans,"[CNA00300, CNN02320, CNE02100, CNA05950, CNF04...","{'1': 0, '2': 2300533, '3': 3932840, '4': 6038..."
6,Nasakeomyces glabrata,"[CAGL0E04334g, CAGL0F01793g, CAGL0G01034g, CAG...","{'ChrA_C_glabrata_CBS138': 0, 'ChrB_C_glabrata..."
7,Pichia kudriavzevii,"[JL09_g2508, JL09_g3074, JL09_g1956, JL09_g200...","{'scaffold00001': 0, 'scaffold00002': 607893, ..."


In [6]:
df.to_csv('landmarks.csv')

## Get gene sequences
synth : the string of appended gene sequences, spacers are placed in between the different genes 

In [10]:
synth = ""
spacer = "NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN"

for i, file in enumerate(species_genome_file_list):
    # get species associated to the ref genome file,
    # add header to synthetic genome.
    species = species_list[i]
    sp = re.sub(" ", "", species)
    print(sp)
    synth += f">{sp}\n"
    
    with open(file) as genome:
        genome_content = genome.read()
    
    # Removes header lines and removes break lines.
    dna = re.sub(">.*", "", genome_content)
    dna = re.sub("\n", "", dna)
    
    # read gff file associated to the ref genome file.
    gff_file = gff_files_list[i]
    with open(gff_file) as gff:
        gff_content = gff.read()
    
    # get gene ids from the dataframe created earlier in the code.
    gene_list = df['Gene ID'][df['Species'] == species].values[0]
    
    for g in gene_list:
        
        #print(g)
        # find gene coordinates
        y = re.findall(f"^.*\t.*\tgene\t.*?{g}", gff_content, flags=re.MULTILINE)
        coordinates = re.findall("[^\\t]+", y[0])
        print(coordinates)
        index = coordinates[0]
        
        # get associated landmark
        thelandmark = df['Landmarks'][df['Species'] == species].values[0][index]

        begin = int(coordinates[3]) + int(thelandmark) -1 # Minus 1 to get to base 0 
        end = int(coordinates[4]) + int(thelandmark) # Here we do not substract because we want to include the last nucleotide.
        gene_seq = dna[begin:end]
        
        print(f"{begin} --- {end}")
        print(gene_seq)
        
        synth += gene_seq + spacer * 5
    
    synth += '\n'

Aspergillusfumigatus
['4', 'ena', 'gene', '1780204', '1781822', '.', '-', '.', 'ID=gene:AFUA_4G06890']
15622821 --- 15624440
TCACTTGGATGTGTTTTTCGACCGCTTCTCCCAGCCGATGATGCTTGGCTTCATGGGGCCCGAAAAGAGGGATGAATAGTCAGTTTCAGGGACTCCTTTCTTTCCATCCACGTTGAAAAGTCGCAGGTGGCGCACAATGGTCGCCAGAATCACACCAAGGTTGACATAAGCGAATTTCTCGCCGATGCAGCGGTGTCGGCCAGCACCAAACGGAAGATAGGGACTTGACGTGCCCTTGGAGACGGCGCCGTAACCGTAGTCGACAACCTTGTCGTTCTCCTGCTCCTTAGTAGCCTGGTTCTCCCAGCGATGGGGATCCCAGCACCCAGCATTGGGGAAGTGTTCGTCGCTGAGGGCTGTCACTCCAGGTGAAGCAAGGAGCACGCGACCGGGAGGAATCATGTAAGGGGTCCCGGGAACGGGCAAGGGGCTTTTCACCTTGCGCATGATAGAGTGAATAGAGGAGTGAATCCGCAAGGTTTCACGAATAACATGTTGATGGAAGGGAAGTTTGTCAAGATCCTTGTACTGGAGCGGAGGAAGACTGCCGTCTGGCCCGGCGGGGCCAAGATTGGCCAGCTGTTCCTGATACAGCTCTTCGAGGACTTTTGGCTGTGAGGCCAGTCTCAGCATAATCCAGGCGCTGATGGACGAAGACGAATGCTGACCAGCCATCAACAGGGTTATCATCATGTGCGCAATCTCTTTATCAGGCACTTGCTGGCCGTTTTTGTATGTGCAGTTCATCAGGTTCCATATCATGTCTGATTTCTGAGAGTCCTTCTCACCGTCAAGACGGCGCTGAGTGATGATGTCAACGTAGATTGACCTCATGCGCGCATGAGCAGCATCTCGCTTCTTGTTATGCGGCAATG

## Fasta file creation

In [ ]:
# Fasta files have 60 characters-lines
synth_w_newlines = re.sub("([A-Za-z]{60})", "\\1\n", synth, 0, re.DOTALL)

#my_synth_genome = open("v4_synthetic_genome.fa", "w")
#my_synth_genome.write(synth_w_newlines)
#my_synth_genome.close()